In [1]:
import sys
import os

notebook_path = os.getcwd() 
parent_dir = os.path.dirname(notebook_path)
project_root = os.path.dirname(parent_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from copy import deepcopy

In [3]:
import state_NN_models
import Filters
import utils
import Systems
from utils import losses, trainer, utils
from torch.utils.data import TensorDataset, DataLoader, random_split
from state_NN_models.StateBayesianKalmanNet import StateBayesianKalmanNet
from state_NN_models.StateKalmanNet import StateKalmanNet
from state_NN_models.StateKalmanNetWithKnownR import StateKalmanNetWithKnownR

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používané zařízení: {device}")

Používané zařízení: cuda


In [5]:
import torch
from math import pi
import Systems

# =================================================================================
# KROK 1: DEFINICE PARAMETRŮ PRO NELINEÁRNÍ SYSTÉM ("Synthetic")
# =================================================================================
state_dim_nl = 2
obs_dim_nl = 2

# Používáme pouze parametry "Ground Truth"
alpha_true = 0.9
beta_true = 1.1
phi_true = 0.1 * pi
delta_true = 0.01
a_true = 1.0
b_true = 1.0
c_true = 0.0

f_true_nonlinear = lambda x: alpha_true * torch.sin(beta_true * x + phi_true) + delta_true
h_true_nonlinear = lambda x: a_true * (b_true * x + c_true)**2

Q_true_nl = torch.eye(state_dim_nl) * 0.5
R_true_nl = torch.eye(obs_dim_nl) * 0.1
Ex0_true_nl = torch.tensor([[1.0], [0.0]])
P0_true_nl = torch.eye(state_dim_nl) * 1.5

# =================================================================================
# KROK 2: INICIALIZACE OBJEKTU SYSTÉMU S GAUSSOVSKÝM ŠUMEM
# =================================================================================
print("\nInicializuji 2D 'Synthetic' nelineární systém (Čistý Gaussovský šum)...")

# Ideální reálný systém
sys_true = Systems.DynamicSystem(
    state_dim=state_dim_nl, obs_dim=obs_dim_nl,
    Ex0=Ex0_true_nl, P0=P0_true_nl,
    Q=Q_true_nl, R=R_true_nl,
    f=f_true_nonlinear, h=h_true_nonlinear,
    device=device,
    noise_type='gaussian' # Fáze 1: Pouze standardní Gaussovský šum
)

# Zásadní krok pro tento experiment: Filtrům dáme PŘESNOU kopii reality
sys_model = sys_true

print("... Nelineární systém inicializován (sys_model = sys_true).")


Inicializuji 2D 'Synthetic' nelineární systém (Čistý Gaussovský šum)...
... Nelineární systém inicializován (sys_model = sys_true).


In [6]:
TRAIN_SEQ_LEN = 10      # Krátké sekvence pro stabilní trénink (TBPTT)
VALID_SEQ_LEN = 20      # Stejná délka pro konzistentní validaci
TEST_SEQ_LEN = 100      # Dlouhé sekvence pro testování generalizace

NUM_TRAIN_TRAJ = 1000   # Hodně trénovacích příkladů
NUM_VALID_TRAJ = 200    # Dostatek pro spolehlivou validaci
NUM_TEST_TRAJ = 100     # Pro robustní vyhodnocení

BATCH_SIZE = 64         # Dobrý kompromis

x_train, y_train = utils.generate_data(sys_true, num_trajectories=NUM_TRAIN_TRAJ, seq_len=TRAIN_SEQ_LEN)
x_val, y_val = utils.generate_data(sys_true, num_trajectories=NUM_VALID_TRAJ, seq_len=VALID_SEQ_LEN)
x_test, y_test = utils.generate_data(sys_true, num_trajectories=1, seq_len=TEST_SEQ_LEN)

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import os
import random
import csv
from datetime import datetime
import pandas as pd
from copy import deepcopy

model_config = {
    "hidden_size_multiplier": 12,
    "output_layer_multiplier": 4,
    "num_gru_layers": 1,
    "init_min_dropout": 0.5,
    "init_max_dropout": 0.8
}

train_config = {
    "total_train_iter": 1400,
    "learning_rate": 1e-4,
    "clip_grad": 10.0,
    "J_samples": 10,
    "validation_period": 10,
    "logging_period": 20,
    "warmup_iterations":0, # Trénuj prvních 400 iterací jen na MSE
    "store_model_based_on_hybrid_score": True,
    "calibration_parameter": 0.15
}
# =================================================================================
# KROK 3: SPUŠTĚNÍ JEDNOHO TRÉNINKOVÉHO BĚHU NEBO NAČTENÍ HOTOVÉHO MODELU
# =================================================================================
print(f"Používané zařízení: {device}")
print("="*80)
print("Připravuji Bayesian KalmanNet (BKN)...")
print(f"Parametry modelu: {model_config}")
print(f"Parametry tréninku: {train_config}")
print("="*80)

# Nastavení seedu pro reprodukovatelnost tohoto běhu
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Definice cesty ke složce a souboru
save_dir = "NonLinearSystemWeights"
os.makedirs(save_dir, exist_ok=True)
weights_path = os.path.join(save_dir, "best_bkn_model_weights.pth")

# Vytvoření modelu (potřebujeme ho inicializovat v obou případech)
state_bkn_knet = StateBayesianKalmanNet(
    sys_model,
    device=device,
    weight_init=False,
    **model_config
).to(device)

# Podmínka pro načtení nebo trénink
if os.path.exists(weights_path):
    print(f"\nINFO: Uložené váhy nalezeny v '{weights_path}'.")
    print("Přeskakuji trénink a načítám hotový model...")
    
    # Načtení vah
    state_bkn_knet.load_state_dict(torch.load(weights_path, map_location=device))
    trained_model = state_bkn_knet
    
else:
    print("\nINFO: Váhy nebyly nalezeny. Spouštím plnohodnotný tréninkový běh...")
    
    # Spuštění tréninku
    results = trainer.training_session_trajectory_with_gaussian_nll_training_fcn(
        model=state_bkn_knet,
        train_loader=train_loader,
        val_loader=val_loader,
        device=device,
        **train_config
    )

    # `run_training_session` automaticky načte nejlepší model zpět,
    # takže `state_bkn_knet` nyní obsahuje váhy nejlepšího modelu.
    trained_model = results['final_model']
    
    # Uložení vah pro příští použití
    torch.save(trained_model.state_dict(), weights_path)
    print(f"\nINFO: Váhy modelu byly úspěšně uloženy do: {weights_path}")

    # Výpis metrik pouze pokud jsme trénovali
    print("\n" + "="*80)
    print("TRÉNINK DOKONČEN - FINÁLNÍ VÝSLEDKY Z NEJLEPŠÍHO MODELU")
    print("="*80)
    print(f"Nejlepší model byl nalezen v iteraci: {results['best_iter']}")
    print(f"Nejlepší dosažený validační ANEES: {results['best_val_anees']:.4f}")
    print("--- Metriky odpovídající tomuto nejlepšímu modelu ---")
    print(f"  MSE na validační sadě:       {results['best_val_mse']:.4f}")
    print(f"  NLL na validační sadě:       {results['best_val_nll']:.4f}")
    print("="*80)

# Prepnutí do evaluačního módu na konci bloku (klíčové pro BKN kvůli Dropoutu)
trained_model.eval()
print("\nModel BKN je plně připraven v proměnné 'trained_model'.")

Připravuji Bayesian KalmanNet (BKN)...
Parametry modelu: {'hidden_size_multiplier': 12, 'output_layer_multiplier': 4, 'num_gru_layers': 1, 'init_min_dropout': 0.5, 'init_max_dropout': 0.8}
Parametry tréninku: {'total_train_iter': 1400, 'learning_rate': 0.0001, 'clip_grad': 10.0, 'J_samples': 10, 'validation_period': 10, 'logging_period': 20, 'warmup_iterations': 0, 'store_model_based_on_hybrid_score': True, 'calibration_parameter': 0.15}
loaded with input normalization

INFO: Váhy nebyly nalezeny. Spouštím plnohodnotný tréninkový běh...

--- Validation at iteration 10 ---
  Average MSE: 0.9625, Average ANEES: 54.8446
  Hybrid Score: 8.8892 (Penalty: 52.8446 * 0.15)
  >>> New best HYBRID SCORE! (8.8892 < inf) Saving model. <<<
--------------------------------------------------
--- Iteration [20/1400] ---
    - Total Loss: 13.1449
    - NLL: 13.1414
    - Reg: 0.0034
    - p1=0.628, p2=0.722

--- Validation at iteration 20 ---
  Average MSE: 0.9875, Average ANEES: 44.2460
  Hybrid Score:

In [8]:

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import os
import random
import csv
from datetime import datetime
import pandas as pd
from copy import deepcopy
# Nastavení seedu pro reprodukovatelnost tohoto běhu
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

save_dir = "NonLinearSystemWeights"
weights_path = os.path.join(save_dir, "best_knet_model_weights.pth")

# Vytvoření cílové složky (pokud ještě neexistuje, nic se nestane)
os.makedirs(save_dir, exist_ok=True)


state_knetR = StateKalmanNet(sys_model, device=device, hidden_size_multiplier=12,returns_covariance=True).to(device)

if os.path.exists(weights_path):
    print(f"INFO: Uložené váhy nalezeny v '{weights_path}'.")
    print("Přeskakuji trénink a načítám hotový model...")
    
    # Načtení vah (map_location zajistí kompatibilitu, pokud bys např. trénoval na GPU a načítal na CPU)
    state_knetR.load_state_dict(torch.load(weights_path, map_location=device))
    
else:
    print("INFO: Váhy nebyly nalezeny. Spouštím trénink modelu...")
    
    # Spuštění tréninku
    trainer.train_state_KalmanNet(
        model=state_knetR, 
        train_loader=train_loader, 
        val_loader=val_loader, 
        device=device, 
        epochs=100, 
        lr=1e-4,
        early_stopping_patience=30,
        weight_decay=1e-3
    )
    
    # Po úspěšném tréninku váhy rovnou uložíme pro příští spuštění
    torch.save(state_knetR.state_dict(), weights_path)
    print(f"Trénink dokončen. Váhy byly bezpečně uloženy do '{weights_path}'.")

# Prepnutí do evaluačního módu (dobrá praxe před testováním)
state_knetR.eval()

DEBUG: Layer 'output_final_linear.0' initialized near zero (Start K=0).
INFO: Váhy nebyly nalezeny. Spouštím trénink modelu...
New best model saved! Epoch [1/100], Train Loss: 0.840027, Val Loss: 0.864189
New best model saved! Epoch [2/100], Train Loss: 0.828516, Val Loss: 0.850417
New best model saved! Epoch [3/100], Train Loss: 0.817975, Val Loss: 0.835192
New best model saved! Epoch [4/100], Train Loss: 0.803908, Val Loss: 0.818393
Epoch [5/100] | Train Loss: 0.7926 (Pos: 0.00, Vel: 0.00) | Val Loss: 0.8012 | Val MSE: 0.80
Epoch [4/100], Train Loss: 0.803908, Val Loss: 0.818393, Avg Cov Trace: 0.013665
New best model saved! Epoch [5/100], Train Loss: 0.792649, Val Loss: 0.801221
New best model saved! Epoch [6/100], Train Loss: 0.773086, Val Loss: 0.783724
New best model saved! Epoch [7/100], Train Loss: 0.763495, Val Loss: 0.765168
New best model saved! Epoch [8/100], Train Loss: 0.746346, Val Loss: 0.747011
New best model saved! Epoch [9/100], Train Loss: 0.726362, Val Loss: 0.7297

StateKalmanNet(
  (dnn): DNN_KalmanNet(
    (input_layer): Sequential(
      (0): Linear(in_features=8, out_features=384, bias=True)
      (1): ReLU()
    )
    (gru): GRU(384, 8)
    (output_hidden_layer): Sequential(
      (0): Linear(in_features=8, out_features=16, bias=True)
      (1): ReLU()
    )
    (output_final_linear): Sequential(
      (0): Linear(in_features=16, out_features=4, bias=True)
    )
  )
)

In [9]:
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import Filters

# ==============================================================================
# 0. PŘEDPOKLADY - ZDE PŘIŘAĎTE VAŠE NATRÉNOVANÉ MODELY
# ==============================================================================
try:
    trained_model_bkn = trained_model
    trained_model_knetR = state_knetR
    print("INFO: Všechny natrénované modely nalezeny a přiřazeny.")
except NameError:
    print("VAROVÁNÍ: Některé z proměnných `trained_model`, nebo `state_knetR` nebyly nalezeny.")
    print("         Ujistěte se, že jste nejprve úspěšně dokončili trénink všech modelů.")

trained_model_knetR.returns_covariance=True

# ==============================================================================
# 1. KONFIGURACE TESTU
# ==============================================================================
TEST_SEQ_LEN = 300
NUM_TEST_TRAJ = 30
J_SAMPLES_TEST = 25

# ==============================================================================
# 2. PŘÍPRAVA DAT (Nelineární Gaussovský systém)
# ==============================================================================
print(f"\nGeneruji {NUM_TEST_TRAJ} testovacích trajektorií o délce {TEST_SEQ_LEN}...")
x_test, y_test = utils.generate_data(sys_true, num_trajectories=NUM_TEST_TRAJ, seq_len=TEST_SEQ_LEN)
test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
print("Generování dat dokončeno.")

# ==============================================================================
# 3. INICIALIZACE VŠECH FILTRŮ PRO POROVNÁNÍ (IDEÁLNÍ MODEL)
# ==============================================================================
ekf_ideal = Filters.ExtendedKalmanFilter(sys_true)
ukf_ideal = Filters.UnscentedKalmanFilter(sys_true)
# Particle filtr s vysokým počtem částic představuje "zlatý standard" pro nelinearity
pf_ideal = Filters.VectorizedParticleFilter(sys_true, num_particles=100000)

print("Všechny model-based filtry (EKF, UKF, PF) s ideálním modelem inicializovány.")

# ==============================================================================
# 4. VYHODNOCOVACÍ SMYČKA
# ==============================================================================
all_x_true_cpu = []
all_x_hat_bkn_cpu, all_P_hat_bkn_cpu = [], []
all_x_hat_knetR_cpu, all_P_hat_knetR_cpu = [], []
all_x_hat_ekf_ideal_cpu, all_P_hat_ekf_ideal_cpu = [], []
all_x_hat_ukf_ideal_cpu, all_P_hat_ukf_ideal_cpu = [], []
all_x_hat_pf_ideal_cpu, all_P_hat_pf_ideal_cpu = [], []

print(f"\nVyhodnocuji modely na {NUM_TEST_TRAJ} testovacích trajektoriích...")

trained_model_bkn.eval() 
trained_model_knetR.eval()

with torch.no_grad():
    for i, (x_true_seq_batch, y_test_seq_batch) in enumerate(test_loader):
        y_test_seq_gpu = y_test_seq_batch.squeeze(0).to(device)
        x_true_seq_gpu = x_true_seq_batch.squeeze(0).to(device)
        initial_state = x_true_seq_gpu[0, :].unsqueeze(0)
        
        # --- A. Bayesian KalmanNet (Trajectory-wise) ---
        ensemble_trajectories = []
        for j in range(J_SAMPLES_TEST):
            trained_model_bkn.reset(batch_size=1, initial_state=initial_state)
            current_x_hats = []
            for t in range(1, TEST_SEQ_LEN):
                x_filtered_t, _ = trained_model_bkn.step(y_test_seq_gpu[t, :].unsqueeze(0))
                current_x_hats.append(x_filtered_t)
            ensemble_trajectories.append(torch.cat(current_x_hats, dim=0))
        ensemble = torch.stack(ensemble_trajectories, dim=0)
        predictions_bkn = ensemble.mean(dim=0)
        diff = ensemble - predictions_bkn.unsqueeze(0)
        covariances_bkn = (diff.unsqueeze(-1) @ diff.unsqueeze(-2)).mean(dim=0)
        full_x_hat_bkn = torch.cat([initial_state, predictions_bkn], dim=0)
        full_P_hat_bkn = torch.cat([sys_true.P0.unsqueeze(0), covariances_bkn], dim=0)

        # --- B. StateKalmanNetWithKnownR ---
        trained_model_knetR.reset(batch_size=1, initial_state=initial_state)
        knetR_preds_x, knetR_preds_P = [], []
        for t in range(1, TEST_SEQ_LEN):
            x_filtered_t, P_filtered_t = trained_model_knetR.step(y_test_seq_gpu[t, :].unsqueeze(0))
            knetR_preds_x.append(x_filtered_t)
            knetR_preds_P.append(P_filtered_t)
        full_x_hat_knetR = torch.cat([initial_state, torch.cat(knetR_preds_x, dim=0)], dim=0)
        
        processed_P_list = []
        for p_tensor in knetR_preds_P:
            while p_tensor.dim() < 2:
                p_tensor = p_tensor.unsqueeze(-1)
            if p_tensor.dim() > 2 and p_tensor.shape[0] == 1:
                p_tensor = p_tensor.squeeze(0)
            processed_P_list.append(p_tensor)

        P_sequence_knetR = torch.stack(processed_P_list, dim=0)
        
        P0_for_cat = sys_true.P0.clone()
        while P0_for_cat.dim() < P_sequence_knetR.dim():
            P0_for_cat = P0_for_cat.unsqueeze(0)
            
        full_P_hat_knetR = torch.cat([P0_for_cat, P_sequence_knetR], dim=0)

        # --- C. EKF (Ideální) ---
        ekf_i_res = ekf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)
        full_x_hat_ekf_i = ekf_i_res['x_filtered']
        full_P_hat_ekf_i = ekf_i_res['P_filtered']

        # --- D. UKF (Ideální) ---
        ukf_i_res = ukf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)
        full_x_hat_ukf_i = ukf_i_res['x_filtered']
        full_P_hat_ukf_i = ukf_i_res['P_filtered']

        # --- E. Particle Filter (Ideální) ---
        pf_i_res = pf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)
        full_x_hat_pf_i = pf_i_res['x_filtered']
        full_P_hat_pf_i = pf_i_res['P_filtered']

        # Ukládání pro výpočet metrik
        all_x_true_cpu.append(x_true_seq_gpu.cpu())
        all_x_hat_bkn_cpu.append(full_x_hat_bkn.cpu()); all_P_hat_bkn_cpu.append(full_P_hat_bkn.cpu())
        all_x_hat_knetR_cpu.append(full_x_hat_knetR.cpu()); all_P_hat_knetR_cpu.append(full_P_hat_knetR.cpu())
        all_x_hat_ekf_ideal_cpu.append(full_x_hat_ekf_i.cpu()); all_P_hat_ekf_ideal_cpu.append(full_P_hat_ekf_i.cpu())
        all_x_hat_ukf_ideal_cpu.append(full_x_hat_ukf_i.cpu()); all_P_hat_ukf_ideal_cpu.append(full_P_hat_ukf_i.cpu())
        all_x_hat_pf_ideal_cpu.append(full_x_hat_pf_i.cpu()); all_P_hat_pf_ideal_cpu.append(full_P_hat_pf_i.cpu())

        print(f"Dokončena trajektorie {i + 1}/{NUM_TEST_TRAJ}...")

# ==============================================================================
# 5. FINÁLNÍ VÝPOČET A VÝPIS METRIK
# ==============================================================================
# Seznamy pro sběr metrik
mse_bkn, anees_bkn = [], []
mse_knetR, anees_knetR = [], []
mse_ekf_ideal, anees_ekf_ideal = [], []
mse_ukf_ideal, anees_ukf_ideal = [], []
mse_pf_ideal, anees_pf_ideal = [], []

print("\nPočítám finální metriky pro jednotlivé trajektorie...")

with torch.no_grad():
    for i in range(NUM_TEST_TRAJ):
        x_true = all_x_true_cpu[i]
        
        def get_metrics(x_hat, P_hat):
            mse = F.mse_loss(x_hat[1:], x_true[1:]).item()
            anees = utils.calculate_anees_vectorized(x_true.unsqueeze(0), x_hat.unsqueeze(0), P_hat.unsqueeze(0))
            return mse, anees

        # Výpočty pro všechny modely
        mse, anees = get_metrics(all_x_hat_bkn_cpu[i], all_P_hat_bkn_cpu[i]); mse_bkn.append(mse); anees_bkn.append(anees)
        mse, anees = get_metrics(all_x_hat_knetR_cpu[i], all_P_hat_knetR_cpu[i]); mse_knetR.append(mse); anees_knetR.append(anees)
        mse, anees = get_metrics(all_x_hat_ekf_ideal_cpu[i], all_P_hat_ekf_ideal_cpu[i]); mse_ekf_ideal.append(mse); anees_ekf_ideal.append(anees)
        mse, anees = get_metrics(all_x_hat_ukf_ideal_cpu[i], all_P_hat_ukf_ideal_cpu[i]); mse_ukf_ideal.append(mse); anees_ukf_ideal.append(anees)
        mse, anees = get_metrics(all_x_hat_pf_ideal_cpu[i], all_P_hat_pf_ideal_cpu[i]); mse_pf_ideal.append(mse); anees_pf_ideal.append(anees)

# Funkce pro bezpečné průměrování
def avg(metric_list): return np.mean([m for m in metric_list if not np.isnan(m)])
state_dim_for_nees = all_x_true_cpu[0].shape[1]

# --- Finální výpis tabulky ---
print("\n" + "="*85)
print(f"FINÁLNÍ VÝSLEDKY: FÁZE 1 - NELINEARITA BEZ MISMATCHU (průměr přes {NUM_TEST_TRAJ} běhů)")
print("="*85)
print(f"{'Model':<35} | {'Průměrné MSE':<20} | {'Průměrný ANEES':<20}")
print("-" * 85)
print(f"{'--- Data-Driven Models ---':<35} | {'(nižší je lepší)':<20} | {'(bližší ' + str(float(state_dim_for_nees)) + ' je lepší)':<20}")
print(f"{'Bayesian KNet (BKN)':<35} | {avg(mse_bkn):<20.4f} | {avg(anees_bkn):<20.4f}")
print(f"{'KNet with Known R (KNetR)':<35} | {avg(mse_knetR):<20.4f} | {avg(anees_knetR):<20.4f}")
print("-" * 85)
print(f"{'--- Model-Based Filters (Ideální) ---':<35} | {'':<20} | {'':<20}")
print(f"{'EKF (Ideální model)':<35} | {avg(mse_ekf_ideal):<20.4f} | {avg(anees_ekf_ideal):<20.4f}")
print(f"{'UKF (Ideální model)':<35} | {avg(mse_ukf_ideal):<20.4f} | {avg(anees_ukf_ideal):<20.4f}")
print(f"{'Particle Filter (Ideální, 100k)':<35} | {avg(mse_pf_ideal):<20.4f} | {avg(anees_pf_ideal):<20.4f}")
print("="*85)

INFO: Všechny natrénované modely nalezeny a přiřazeny.

Generuji 30 testovacích trajektorií o délce 300...
Generování dat dokončeno.
Všechny model-based filtry (EKF, UKF, PF) s ideálním modelem inicializovány.

Vyhodnocuji modely na 30 testovacích trajektoriích...


Processing PF (GPU): 100%|██████████| 299/299 [00:00<00:00, 522.38it/s]


Dokončena trajektorie 1/30...


Processing PF (GPU): 100%|██████████| 299/299 [00:00<00:00, 558.32it/s]


Dokončena trajektorie 2/30...


...

Dokončena trajektorie 28/30...


Processing PF (GPU): 100%|██████████| 299/299 [00:00<00:00, 558.56it/s]


Dokončena trajektorie 29/30...


Processing PF (GPU): 100%|██████████| 299/299 [00:00<00:00, 552.73it/s]

Dokončena trajektorie 30/30...

Počítám finální metriky pro jednotlivé trajektorie...

FINÁLNÍ VÝSLEDKY: FÁZE 1 - NELINEARITA BEZ MISMATCHU (průměr přes 30 běhů)
Model                               | Průměrné MSE         | Průměrný ANEES      
-------------------------------------------------------------------------------------
--- Data-Driven Models ---          | (nižší je lepší)     | (bližší 2.0 je lepší)
Bayesian KNet (BKN)                 | 0.7509               | 2.2397              
KNet with Known R (KNetR)           | 0.6680               | 122.0965            
-------------------------------------------------------------------------------------
--- Model-Based Filters (Ideální) --- |                      |                     
EKF (Ideální model)                 | 1.8416               | 61.9660             
UKF (Ideální model)                 | 0.7464               | 5.6536              
Particle Filter (Ideální, 100k)     | 0.5980               | 852.5483            
